In [ ]:
def load_data():
    """
    Función utilitaria para cargar y preparar los datos del dataset.
    Esta función centraliza la carga de datos para evitar duplicación de código.
    """

    # ============================================================================
    # IMPORTACIÓN DE LIBRERÍA PARA MANEJO DE DATOS
    # ============================================================================
    import pandas as pd  # Librería principal para manipulación de datos estructurados

    # ============================================================================
    # CARGA DEL DATASET DESDE ARCHIVO COMPRIMIDO
    # ============================================================================
    
    # Cargar el dataset de frases desde un archivo CSV comprimido
    # Utiliza ruta relativa desde la ubicación del notebook
    dataframe = pd.read_csv(
        "../files/input/sentences.csv.zip",  # Ruta relativa al archivo de datos comprimido
        index_col=False,  # No usar ninguna columna como índice del DataFrame
        compression="zip",  # Especificar que el archivo está en formato ZIP
    )

    # ============================================================================
    # SEPARACIÓN DE CARACTERÍSTICAS Y ETIQUETAS
    # ============================================================================
    
    # Extraer las frases de texto que serán las características (features) del modelo
    data = dataframe.phrase  # Columna con el texto a clasificar
    
    # Extraer las etiquetas objetivo que representan las clases verdaderas
    target = dataframe.target  # Columna con las categorías/clases de cada frase

    # ============================================================================
    # RETORNO DE DATOS SEPARADOS
    # ============================================================================
    
    # Devolver por separado las características y las etiquetas
    # Esto facilita su uso en otras funciones del pipeline
    return data, target

In [ ]:
def make_train_test_split(x, y):
    """
    Función que divide los datos en conjuntos de entrenamiento y prueba.
    Utiliza una semilla aleatoria para garantizar reproducibilidad de resultados.
    
    Args:
        x: Características/features (frases de texto)
        y: Etiquetas/targets (categorías de clasificación)
    """

    # ============================================================================
    # IMPORTACIÓN DE FUNCIÓN PARA DIVISIÓN DE DATOS
    # ============================================================================
    from sklearn.model_selection import train_test_split  # Función para dividir datasets

    # ============================================================================
    # DIVISIÓN ESTRATIFICADA DE LOS DATOS
    # ============================================================================
    
    # Dividir el dataset en conjuntos de entrenamiento (75%) y prueba (25%)
    # La división es aleatoria pero reproducible gracias a random_state
    (x_train, x_test, y_train, y_test) = train_test_split(
        x,  # Datos de entrada (frases de texto)
        y,  # Etiquetas correspondientes
        test_size=0.25,  # 25% de los datos para conjunto de prueba
        random_state=123456,  # Semilla para reproducibilidad de la división aleatoria
    )
    
    # ============================================================================
    # RETORNO DE CONJUNTOS DIVIDIDOS
    # ============================================================================
    
    # Devolver los cuatro conjuntos resultantes:
    # - x_train, y_train: para entrenar el modelo
    # - x_test, y_test: para evaluar el rendimiento del modelo
    return x_train, x_test, y_train, y_test

In [ ]:
def make_pipeline(estimator):
    """
    Función que construye un pipeline de procesamiento de texto completo.
    Crea una secuencia de transformaciones que preparan el texto para clasificación.
    
    Args:
        estimator: Algoritmo de machine learning a usar (ej: LogisticRegression, MLPClassifier)
    """

    # ============================================================================
    # IMPORTACIÓN DE COMPONENTES DEL PIPELINE
    # ============================================================================
    from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer  # Para procesamiento de texto
    from sklearn.pipeline import Pipeline  # Para crear secuencias de transformaciones

    # ============================================================================
    # CONFIGURACIÓN DEL VECTORIZADOR DE TEXTO
    # ============================================================================
    
    # Primer componente: Convierte texto crudo en matriz numérica de conteos
    vectorizer = CountVectorizer(
        lowercase=True,  # Normalizar texto convirtiendo a minúsculas
        analyzer="word",  # Procesar a nivel de palabras individuales
        token_pattern=r"\b[a-zA-Z]\w+\b",  # Regex: palabras que empiecen con letra y contengan alfanuméricos
        stop_words="english",  # Filtrar palabras comunes del inglés (the, and, is, etc.)
    )

    # ============================================================================
    # CONFIGURACIÓN DEL TRANSFORMADOR TF-IDF
    # ============================================================================
    
    # Segundo componente: Convierte conteos brutos en pesos TF-IDF
    # TF-IDF da mayor peso a palabras discriminativas y menor a palabras comunes
    transformer = TfidfTransformer()

    # ============================================================================
    # CONSTRUCCIÓN DEL PIPELINE SECUENCIAL
    # ============================================================================
    
    # Crear pipeline que ejecuta transformaciones en orden secuencial:
    # texto → conteos → TF-IDF → clasificación
    pipeline = Pipeline(
        steps=[
            ("vectorizer", vectorizer),  # Paso 1: Convertir texto a matriz de conteos
            ("transformer", transformer),  # Paso 2: Aplicar transformación TF-IDF
            ("estimator", estimator),  # Paso 3: Aplicar algoritmo de clasificación
        ],
        verbose=False,  # No mostrar información detallada durante ejecución
    )

    # ============================================================================
    # RETORNO DEL PIPELINE CONFIGURADO
    # ============================================================================
    
    # Devolver el pipeline completo listo para entrenamiento
    return pipeline

In [ ]:
def make_grid_search(estimator, param_grid, cv=5):
    """
    Función que configura búsqueda exhaustiva de hiperparámetros.
    Utiliza validación cruzada para encontrar la mejor combinación de parámetros.
    
    Args:
        estimator: Pipeline o modelo a optimizar
        param_grid: Diccionario con parámetros y valores a probar
        cv: Número de folds para validación cruzada (default=5)
    """

    # ============================================================================
    # IMPORTACIÓN DE HERRAMIENTA PARA BÚSQUEDA DE HIPERPARÁMETROS
    # ============================================================================
    from sklearn.model_selection import GridSearchCV  # Para búsqueda exhaustiva de parámetros

    # ============================================================================
    # CONFIGURACIÓN DE LA BÚSQUEDA GRID SEARCH
    # ============================================================================
    
    # Crear objeto GridSearchCV que probará todas las combinaciones de parámetros
    grid_search = GridSearchCV(
        estimator=estimator,  # Modelo/pipeline a optimizar
        param_grid=param_grid,  # Diccionario con parámetros a probar
        cv=cv,  # Número de folds para validación cruzada (5 por defecto)
        scoring="balanced_accuracy",  # Métrica para evaluar combinaciones (balanceada para clases desbalanceadas)
    )

    # ============================================================================
    # RETORNO DEL OBJETO GRID SEARCH CONFIGURADO
    # ============================================================================
    
    # Devolver el objeto GridSearchCV listo para entrenar
    # Al hacer fit(), probará todas las combinaciones y seleccionará la mejor
    return grid_search

In [ ]:
def save_estimator(estimator):
    """
    Función que guarda un modelo entrenado en disco para uso posterior.
    Utiliza serialización binaria para preservar el estado completo del modelo.
    
    Args:
        estimator: Modelo/pipeline entrenado que se desea persistir
    """

    # ============================================================================
    # IMPORTACIÓN DE LIBRERÍA PARA SERIALIZACIÓN
    # ============================================================================
    import pickle  # Librería estándar de Python para serialización de objetos

    # ============================================================================
    # PERSISTENCIA DEL MODELO EN DISCO
    # ============================================================================
    
    # Abrir archivo en modo escritura binaria para guardar el modelo
    # El contexto 'with' garantiza que el archivo se cierre correctamente
    with open("estimator.pickle", "wb") as file:
        # Serializar y escribir el modelo completo al archivo
        # Esto incluye el pipeline, parámetros entrenados y configuraciones
        pickle.dump(estimator, file)

In [ ]:
def load_estimator():
    """
    Función que carga un modelo previamente guardado desde disco.
    Incluye verificación de existencia del archivo para evitar errores.
    
    Returns:
        estimator: Modelo cargado desde disco, o None si no existe
    """

    # ============================================================================
    # IMPORTACIÓN DE LIBRERÍAS NECESARIAS
    # ============================================================================
    import os  # Para verificar existencia de archivos
    import pickle  # Para deserializar objetos guardados

    # ============================================================================
    # VERIFICACIÓN DE EXISTENCIA DEL ARCHIVO
    # ============================================================================
    
    # Comprobar si existe un modelo previamente guardado
    # Esto evita errores cuando no hay un modelo previo
    if not os.path.exists("estimator.pickle"):
        return None  # Retornar None si no hay modelo guardado

    # ============================================================================
    # CARGA DEL MODELO DESDE DISCO
    # ============================================================================
    
    # Abrir archivo en modo lectura binaria para cargar el modelo
    with open("estimator.pickle", "rb") as file:
        # Deserializar y cargar el modelo completo desde el archivo
        estimator = pickle.load(file)

    # ============================================================================
    # RETORNO DEL MODELO CARGADO
    # ============================================================================
    
    # Devolver el modelo cargado (incluyendo pipeline y parámetros entrenados)
    return estimator

In [ ]:
def train_logistic_regression():
    """
    Función principal para entrenar regresión logística con optimización de hiperparámetros.
    Implementa búsqueda exhaustiva de parámetros y comparación con modelos previos.
    """

    # ============================================================================
    # IMPORTACIÓN DE ALGORITMO Y MÉTRICAS
    # ============================================================================
    from sklearn.linear_model import LogisticRegression  # Algoritmo de regresión logística
    from sklearn.metrics import balanced_accuracy_score  # Métrica balanceada para evaluación

    # ============================================================================
    # PASO 1: CARGA DE DATOS
    # ============================================================================
    
    # Cargar el dataset completo usando la función utilitaria
    data, target = load_data()

    # ============================================================================
    # PASO 2: DIVISIÓN DE DATOS EN ENTRENAMIENTO Y PRUEBA
    # ============================================================================
    
    # Dividir los datos manteniendo reproducibilidad
    x_train, x_test, y_train, y_test = make_train_test_split(
        x=data,  # Frases de texto como características
        y=target,  # Categorías como etiquetas objetivo
    )

    # ============================================================================
    # PASO 3: CONSTRUCCIÓN DEL PIPELINE BASE
    # ============================================================================
    
    # Crear pipeline completo con regresión logística
    # max_iter=1000: suficientes iteraciones para convergencia
    pipeline = make_pipeline(
        estimator=LogisticRegression(max_iter=1000),
    )

    # ============================================================================
    # PASO 4: CONFIGURACIÓN DE BÚSQUEDA DE HIPERPARÁMETROS
    # ============================================================================
    
    # Crear GridSearchCV para optimizar parámetros del TF-IDF transformer
    estimator = make_grid_search(
        estimator=pipeline,
        param_grid={
            # Parámetros del transformador TF-IDF a optimizar:
            "transformer__norm": ["l1", "l2", None],  # Normalización: L1, L2 o sin normalizar
            "transformer__use_idf": [True, False],  # Usar o no el componente IDF
            "transformer__smooth_idf": [True, False],  # Suavizado del IDF para evitar división por cero
        },
        cv=5,  # Validación cruzada con 5 folds
    )

    # ============================================================================
    # PASO 5: ENTRENAMIENTO CON BÚSQUEDA DE HIPERPARÁMETROS
    # ============================================================================
    
    # Entrenar el modelo probando todas las combinaciones de parámetros
    # GridSearchCV automáticamente selecciona la mejor combinación
    estimator.fit(x_train, y_train)

    # ============================================================================
    # PASO 6: COMPARACIÓN CON MODELO PREVIAMENTE GUARDADO
    # ============================================================================
    
    # Intentar cargar un modelo previamente entrenado para comparación
    best_estimator = load_estimator()

    # Si existe un modelo previo, comparar rendimientos
    if best_estimator is not None:

        # Evaluar el modelo previamente guardado en el conjunto de prueba
        saved_balanced_accuracy = balanced_accuracy_score(
            y_true=y_test, y_pred=best_estimator.predict(x_test)
        )

        # Evaluar el modelo recién entrenado en el conjunto de prueba
        current_balanced_accuracy = balanced_accuracy_score(
            y_true=y_test, y_pred=estimator.predict(x_test)
        )

        # Si el modelo anterior es mejor, mantenerlo
        # Solo actualizar si el nuevo modelo supera al anterior
        if current_balanced_accuracy < saved_balanced_accuracy:
            estimator = best_estimator

    # ============================================================================
    # PASO 7: PERSISTENCIA DEL MEJOR MODELO
    # ============================================================================
    
    # Guardar el mejor modelo (nuevo o previo) para uso futuro
    save_estimator(estimator)

# ============================================================================
# EJECUCIÓN DEL ENTRENAMIENTO DE REGRESIÓN LOGÍSTICA
# ============================================================================

# Ejecutar el proceso completo de entrenamiento con optimización
train_logistic_regression()

In [ ]:
def use_estimator():
    """
    Función que utiliza el mejor modelo entrenado para generar predicciones.
    Implementa la fase de inferencia del proceso de machine learning.
    """

    # ============================================================================
    # IMPORTACIÓN DE LIBRERÍAS NECESARIAS
    # ============================================================================
    import pickle  # Para cargar el modelo serializado desde disco
    import pandas as pd  # Para manejo de datos estructurados

    # ============================================================================
    # CARGA DE DATOS PARA REALIZAR PREDICCIONES
    # ============================================================================
    
    # Cargar el dataset de frases que queremos clasificar
    # En un escenario real, estos serían datos nuevos no vistos durante entrenamiento
    dataframe = pd.read_csv(
        "../files/input/sentences.csv.zip",  # Ruta relativa al archivo de datos
        index_col=False,  # No usar ninguna columna como índice
        compression="zip",  # El archivo está comprimido en formato ZIP
    )

    # ============================================================================
    # EXTRACCIÓN DE CARACTERÍSTICAS PARA PREDICCIÓN
    # ============================================================================
    
    # Extraer únicamente las frases de texto (no necesitamos las etiquetas)
    # Estas serán las características de entrada para el modelo
    data = dataframe.phrase

    # ============================================================================
    # CARGA DEL MEJOR MODELO DESDE DISCO
    # ============================================================================
    
    # Cargar el mejor modelo que fue guardado (puede ser de cualquier algoritmo)
    # Esto incluye el pipeline completo con los mejores hiperparámetros
    with open("estimator.pickle", "rb") as file:
        estimator = pickle.load(file)  # Deserializar el modelo optimizado

    # ============================================================================
    # GENERACIÓN DE PREDICCIONES
    # ============================================================================
    
    # Aplicar el modelo optimizado a los datos para obtener predicciones
    # El pipeline automáticamente ejecuta toda la secuencia optimizada:
    # 1. Vectorización del texto con el vocabulario aprendido
    # 2. Transformación TF-IDF con los mejores parámetros encontrados
    # 3. Clasificación usando el mejor modelo entrenado
    prediction = estimator.predict(data)

    # ============================================================================
    # RETORNO DE PREDICCIONES
    # ============================================================================
    
    # Devolver las predicciones de clase para cada frase
    # Resultado: array con la clase predicha para cada entrada
    return prediction

# ============================================================================
# EJECUCIÓN DEL PROCESO DE INFERENCIA
# ============================================================================

# Ejecutar la función para generar predicciones usando el mejor modelo
use_estimator()

array(['neutral', 'positive', 'positive', ..., 'negative', 'negative',
       'negative'], shape=(2264,), dtype=object)

In [ ]:
def check_estimator():
    """
    Función de diagnóstico que evalúa el rendimiento del mejor modelo guardado.
    Calcula métricas en entrenamiento y prueba para detectar overfitting/underfitting.
    """

    # ============================================================================
    # IMPORTACIÓN DE LIBRERÍAS NECESARIAS
    # ============================================================================
    import pickle  # Para cargar el modelo guardado
    import pandas as pd  # Para manejo de datos
    from sklearn.metrics import accuracy_score, balanced_accuracy_score  # Métricas de evaluación

    # ============================================================================
    # PREPARACIÓN DE DATOS PARA EVALUACIÓN
    # ============================================================================
    
    # Cargar los mismos datos usados durante el entrenamiento
    data, target = load_data()

    # Dividir exactamente igual que durante el entrenamiento para comparación válida
    x_train, x_test, y_train_true, y_test_true = make_train_test_split(
        x=data,  # Mismas características
        y=target,  # Mismas etiquetas
    )

    # ============================================================================
    # CARGA DEL MEJOR MODELO GUARDADO
    # ============================================================================
    
    # Cargar el modelo optimizado desde disco
    with open("estimator.pickle", "rb") as file:
        estimator = pickle.load(file)  # Modelo con mejores hiperparámetros

    # ============================================================================
    # GENERACIÓN DE PREDICCIONES EN AMBOS CONJUNTOS
    # ============================================================================
    
    # Predicciones en el conjunto de entrenamiento
    # Esto nos permite evaluar qué tan bien aprendió de los datos de entrenamiento
    y_train_pred = estimator.predict(x_train)
    
    # Predicciones en el conjunto de prueba
    # Esto nos permite evaluar la capacidad de generalización del modelo
    y_test_pred = estimator.predict(x_test)

    # ============================================================================
    # CÁLCULO DE MÉTRICAS DE EVALUACIÓN
    # ============================================================================
    
    # Métricas en conjunto de entrenamiento
    accuracy_train = round(accuracy_score(y_train_true, y_train_pred), 4)
    balanced_accuracy_train = round(
        balanced_accuracy_score(y_train_true, y_train_pred), 4
    )
    
    # Métricas en conjunto de prueba
    accuracy_test = round(accuracy_score(y_test_true, y_test_pred), 4)
    balanced_accuracy_test = round(balanced_accuracy_score(y_test_true, y_test_pred), 4)

    # ============================================================================
    # REPORTE DE RESULTADOS
    # ============================================================================
    
    # Mostrar el mejor modelo encontrado con sus hiperparámetros optimizados
    print(estimator.best_estimator_, ":", sep="")
    
    # Mostrar métricas: test (train) - formato que permite comparar fácilmente
    # Si train >> test: posible overfitting
    # Si train ≈ test: buen balance bias-varianza
    print(f"  Balanced Accuracy: {balanced_accuracy_test} ({balanced_accuracy_train})")
    print(f"           Accuracy: {accuracy_test} ({accuracy_train})")

# ============================================================================
# EJECUCIÓN DE LA EVALUACIÓN DEL MODELO
# ============================================================================

# Ejecutar la evaluación completa del mejor modelo guardado
check_estimator()

Pipeline(steps=[('vectorizer',
                 CountVectorizer(stop_words='english',
                                 token_pattern='\\b[a-zA-Z]\\w+\\b')),
                ('transformer', TfidfTransformer(norm=None, smooth_idf=False)),
                ('estimator', LogisticRegression(max_iter=1000))]):
  Balanced Accuracy: 0.7165 (0.9962)
           Accuracy: 0.8233 (0.9982)


In [ ]:
def train_mlp_classifier():
    """
    Función para entrenar un clasificador de redes neuronales (MLP) con optimización de hiperparámetros.
    Implementa búsqueda exhaustiva y comparación con el mejor modelo existente.
    """

    # ============================================================================
    # IMPORTACIÓN DE ALGORITMO Y MÉTRICAS
    # ============================================================================
    from sklearn.metrics import balanced_accuracy_score  # Métrica balanceada para evaluación
    from sklearn.neural_network import MLPClassifier  # Clasificador de redes neuronales multicapa

    # ============================================================================
    # PASO 1: CARGA DE DATOS
    # ============================================================================
    
    # Cargar el dataset completo usando la función utilitaria
    data, target = load_data()

    # ============================================================================
    # PASO 2: DIVISIÓN DE DATOS EN ENTRENAMIENTO Y PRUEBA
    # ============================================================================
    
    # Dividir los datos manteniendo reproducibilidad (misma semilla que otros modelos)
    x_train, x_test, y_train, y_test = make_train_test_split(
        x=data,  # Frases de texto como características
        y=target,  # Categorías como etiquetas objetivo
    )

    # ============================================================================
    # PASO 3: CONSTRUCCIÓN DEL PIPELINE CON RED NEURONAL
    # ============================================================================
    
    # Crear pipeline completo con clasificador MLP
    # max_iter=10000: suficientes iteraciones para convergencia de la red neuronal
    pipeline = make_pipeline(
        estimator=MLPClassifier(max_iter=10000),
    )

    # ============================================================================
    # PASO 4: CONFIGURACIÓN EXTENSA DE BÚSQUEDA DE HIPERPARÁMETROS
    # ============================================================================
    
    # Crear GridSearchCV para optimizar múltiples aspectos del modelo
    estimator = make_grid_search(
        estimator=pipeline,
        param_grid={
            # Parámetros del transformador TF-IDF:
            "transformer__norm": ["l1", "l2", None],  # Normalización del TF-IDF
            "transformer__use_idf": [True, False],  # Activar/desactivar componente IDF
            "transformer__smooth_idf": [True, False],  # Suavizado para evitar log(0)
            
            # Parámetros específicos de la red neuronal:
            "estimator__hidden_layer_sizes": [(1,), (5,), (5, 5)],  # Arquitecturas: 1 neurona, 5 neuronas, 2 capas de 5
            "estimator__solver": ["adam"],  # Optimizador Adam (eficiente para problemas medianos)
            "estimator__learning_rate_init": [0.01, 0.001, 0.0001],  # Tasa de aprendizaje inicial
        },
        cv=5,  # Validación cruzada con 5 folds
    )

    # ============================================================================
    # PASO 5: ENTRENAMIENTO CON BÚSQUEDA EXHAUSTIVA
    # ============================================================================
    
    # Entrenar la red neuronal probando todas las combinaciones posibles
    # Esto puede tomar considerablemente más tiempo que regresión logística
    estimator.fit(x_train, y_train)

    # ============================================================================
    # PASO 6: COMPARACIÓN CON MEJOR MODELO EXISTENTE
    # ============================================================================
    
    # Cargar el mejor modelo previamente guardado (puede ser LogisticRegression u otro MLP)
    best_estimator = load_estimator()

    # Si existe un modelo previo, realizar competencia entre modelos
    if best_estimator is not None:

        # Evaluar el modelo anteriormente guardado
        saved_balanced_accuracy = balanced_accuracy_score(
            y_true=y_test, y_pred=best_estimator.predict(x_test)
        )

        # Evaluar el nuevo modelo MLP entrenado
        current_balanced_accuracy = balanced_accuracy_score(
            y_true=y_test, y_pred=estimator.predict(x_test)
        )

        # Conservar el modelo anterior si sigue siendo superior
        # Solo reemplazar si el nuevo MLP supera al modelo existente
        if current_balanced_accuracy < saved_balanced_accuracy:
            estimator = best_estimator

    # ============================================================================
    # PASO 7: PERSISTENCIA DEL MEJOR MODELO GLOBAL
    # ============================================================================
    
    # Guardar el mejor modelo (nuevo MLP o modelo previo) para uso futuro
    save_estimator(estimator)

# ============================================================================
# EJECUCIÓN DEL ENTRENAMIENTO DE RED NEURONAL
# ============================================================================

# Ejecutar el proceso completo de entrenamiento y optimización del MLP
train_mlp_classifier()

c:\Users\Samuel Castaño\Desktop\Fundamentos Analtica 2025-1\Clases\Clase 10\PRE-21-pipelines-SamuCasta\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:787: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [ ]:
# ============================================================================
# EVALUACIÓN FINAL DEL MEJOR MODELO DESPUÉS DE ENTRENAR MLP
# ============================================================================

# Ejecutar diagnóstico final para ver si el MLP mejoró el rendimiento general
# Esto mostrará cuál algoritmo (LogisticRegression vs MLPClassifier) resultó mejor
# y sus métricas finales de rendimiento
check_estimator()

NameError: name 'check_estimator' is not defined